# Google Drive 마이그레이션 테스트 (간단 버전)

이 노트북은 `credentials.json` 없이 Google Colab의 내장 인증을 사용합니다.

## 사용 방법
1. 셀을 순서대로 실행하세요
2. Google 계정으로 로그인하세요
3. 파일 목록을 확인하세요


In [ ]:
# 필요한 라이브러리 설치
%pip install google-api-python-client pandas tqdm

import os
import pandas as pd
from tqdm import tqdm
from datetime import datetime

# Google Colab 내장 인증 사용
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

print("✅ 라이브러리 로드 완료")


In [ ]:
# Google 계정 인증 (credentials.json 불필요)
print("🔐 Google 계정 인증을 시작합니다...")
print("팝업 창에서 Google 계정으로 로그인하세요.")

# Google Colab 내장 인증 사용
auth.authenticate_user()

# Google Drive API 서비스 생성
drive_service = build('drive', 'v3')

print("✅ 인증 완료!")


In [ ]:
# 사용자 정보 조회
print("👤 사용자 정보 조회 중...")

try:
    about = drive_service.about().get(fields='user').execute()
    user_info = about.get('user', {})
    
    print(f"✅ 사용자 정보:")
    print(f"  - 이름: {user_info.get('displayName', 'Unknown')}")
    print(f"  - 이메일: {user_info.get('emailAddress', 'Unknown')}")
    
except Exception as e:
    print(f"❌ 사용자 정보 조회 실패: {e}")


In [ ]:
# 파일 목록 조회
print("📁 파일 목록 조회 중...")

try:
    results = drive_service.files().list(
        pageSize=10,
        fields="nextPageToken, files(id, name, mimeType, size, createdTime, modifiedTime, shared)"
    ).execute()
    
    files = results.get('files', [])
    
    if files:
        print(f"✅ 파일 목록 (최대 10개):")
        
        for i, file in enumerate(files, 1):
            file_size = int(file.get('size', 0)) if file.get('size') else 0
            size_str = f"{file_size/1024:.1f}KB" if file_size > 0 else "폴더"
            shared_str = "🔗" if file.get('shared', False) else "🔒"
            
            print(f"  {i:2d}. {file['name']} ({size_str}) {shared_str}")
            print(f"      ID: {file['id']}")
            print(f"      타입: {file.get('mimeType', 'Unknown')}")
            print(f"      수정일: {file.get('modifiedTime', 'Unknown')}")
            print()
    else:
        print("❌ 파일을 찾을 수 없습니다.")
        
except Exception as e:
    print(f"❌ 파일 목록 조회 실패: {e}")
